In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy 
import tensorflow as tf
import os
import pandas as pd
from utils import proc
import random
import glob
from tqdm import tqdm
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img
from PIL import Image
import cv2

Using TensorFlow backend.


In [19]:
def gen_blur(images, kernels, nstd=2, seeds=None):
    '''
    Input a list of 3d tensors for images and kernels
    '''
    nstd = nstd / 255.

    bsz, h, w, c = tf.convert_to_tensor(images).get_shape().as_list()
    kbsz, kh, kw, kc = tf.convert_to_tensor(kernels).get_shape().as_list()
    assert kbsz==bsz and kc==1

    images = tf.transpose(images, [1,2,0,3]) # h x w x bsz x c
    images = tf.reshape(images, [1,h,w,bsz*c])

    kernels = tf.tile(kernels, [1,1,1,c])
    kernels= tf.transpose(kernels, [1,2,0,3]) # kh x kw x bsz x c
    kernels = tf.reshape(kernels, [kh,kw,bsz*c,1])

    padding = [[0,0], [(kh-1)//2,(kh-1)//2], [(kw-1)//2, (kw-1)//2], [0,0]]
    images = tf.pad(images, padding, 'REFLECT')
    blurs = tf.nn.depthwise_conv2d(images, kernels, [1,1,1,1], 'VALID') # 1 x h x w x bsz*c

    blurs = tf.reshape(blurs, [h,w,bsz,c])
    blurs = tf.transpose(blurs, [2,0,1,3])

    if nstd != 0:
        if seeds is None:
            noise = tf.random_normal([bsz,h,w,c],stddev=nstd)
        else:
            noise = []
            for i in range(bsz):
                n = tf.contrib.stateless.stateless_random_normal([h,w,c], seeds[i], dtype=tf.float32) * nstd
                noise.append(n)
            noise = tf.stack(noise)

        blurs = blurs + noise

    return blurs

### Getting Test Image Paths and Corresponding labels

In [20]:
data_root = 'test_data/test
path = data_root

##################################### getting files paths in each folder ##############################
## get folder paths
folders_paths = []
for entry_name in os.listdir(path):
    entry_path =  os.path.join(path, entry_name)
    if os.path.isdir(entry_path):
        folders_paths.append(entry_path)

# catch Folder names into list
foldernames = []
for entry_name in os.listdir(path):
    entry_path = os.path.join(path, entry_name)
    if os.path.isdir(entry_path):
        foldernames.append(entry_name)

#print(foldernames)

### defining dataframe to store every example into csv againt its class
columns = ['image_path','image_id','label']
test_dfrm1 = pd.DataFrame(columns=columns)
#train.loc[0] = [4,5]

## files names into a list with associated label
#image_path = folder_paths[1]

#rgb_frames = []
i = 0
fold_ind =0
for ind,folder in enumerate(folders_paths):
    for image_name in os.listdir(folder):
        image_path = os.path.join(folder,image_name)
        if os.path.isfile(image_path):
            test_dfrm1.loc[i]=[image_path,image_name,foldernames[ind]]
            i=i+1



In [21]:
img_paths = test_dfrm1['image_path']              # extracting image path
img_labels = test_dfrm1['label']                # extracting image label
img_Ids = test_dfrm1['image_id']             # extracting image_id

### Loading Blur Kernels 

In [6]:
data_root = 'blur_kernels'
path = data_root

##################################### getting path of each kernal ##############################
### defining dataframe to store every example into csv againt its class
kernel_paths = glob.glob(path+'/*.png')

In [7]:
blur = scipy.misc.imread(kernel_paths[0])
ker_shp = blur.shape
ker_shp[0],ker_shp[1]

/home/fahad/miniconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """Entry point for launching an IPython kernel.


(19, 19)

### Bluring Images and storing into seperate folder

In [40]:
data_root_blur = 'test_data/val_128/blur_imgs'
data_root_clean = 'test_data/val_128/clean_imgs'
set_resize = 128
#kernel_shape = 17
i = 0
for image,folder in zip(img_paths,img_labels):
    dest_lab_blur_dir = os.path.join(data_root_blur,folder)
    dest_lab_clean_dir = os.path.join(data_root_clean,folder)
    if not os.path.exists(dest_lab_blur_dir):
        os.makedirs(dest_lab_blur_dir)
    if not os.path.exists(dest_lab_clean_dir):
        os.makedirs(dest_lab_clean_dir)
    v = scipy.misc.imresize(scipy.misc.imread(image),(set_resize,set_resize)).reshape(1,set_resize,set_resize,3).astype('float32')/255
    img_clean =load_img(image)
    img_clean = img_clean.resize((set_resize, set_resize))    
    img_clean.save(os.path.join(dest_lab_clean_dir,str(i)+'.jpg'))
    idx = random.randrange(0,len(kernel_paths))
    blur = scipy.misc.imread(kernel_paths[idx])
    ker_shp = blur.shape
    blur = blur.reshape(1,ker_shp[0],ker_shp[1],1).astype('float32')
    blur = blur/np.sum(blur)
    left_blurs = gen_blur(v, blur, nstd=2, seeds=None)
    sess = tf.Session()
    array = left_blurs.eval(session=sess)
    #im = Image.fromarray(array[0,:,:,:])
    #im.save(os.path.join(dest_lab_dir,str(i)+'.jpg'))
    plt.imsave(os.path.join(dest_lab_blur_dir,str(i)+'.jpg'),np.clip(array[0,:,:,:],0,1))
    i=i+1     
    
    
    #image

/home/fahad/miniconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/home/fahad/miniconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  del sys.path[0]
/home/fahad/miniconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


### Debluring images  by trained deblur model

In [23]:
trained_path = 'wts/supervised'      # set the path where you have saved deblur training

from utils import utils as ut
from utils import pix as net

if trained_path.endswith('.npz'):
    mfile = trained_path
else:
    wts = trained_path
    msave = ut.ckpter(wts + '/iter_*.model.npz')
    mfile = msave.latest

# if out_path is not None:
#     if not os.path.exists(out_path):
#         os.makedirs(out_path)
def csave(fn,img):
    img = np.maximum(0.,np.minimum(1.,img))
    img = np.uint8(img*255.)
    imsave(fn,img)


In [24]:
########################## getting Paths of blur Images ########################3
data_root_blur = 'test_data/val_128/blur_imgs'
data_dir = data_root_blur
# getting folder paths
folder_paths = []
folder_names = []
for fold_name in os.listdir(data_dir):
    fold_path = os.path.join(data_dir,fold_name)
    if os.path.isdir(fold_path):
        folder_paths.append(fold_path)
        folder_names.append(fold_name)

## defining dataframe to store every example into csv
columns =  ['img_id','img_path','label']
blr_dfrm = pd.DataFrame(columns=columns)
i = 0
fold_ind = 0
for ind,folder in enumerate(folder_paths):
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        if os.path.isfile(img_path):
            blr_dfrm.loc[i]=[img_name,img_path,folder_names[ind]]
            i+=1



In [25]:
#### Build Graph
BSZ = len(blr_dfrm)
names, blurs = [], []
for i in range(BSZ):
    nm = tf.placeholder(tf.string)
    blur = tf.read_file(nm)
    blur = tf.image.decode_png(blur, channels=3, dtype=tf.uint8)
    blur = tf.to_float(blur) / 255.
    names.append(nm)
    blurs.append(blur)
blurs = tf.stack(blurs, axis=0)

is_training = tf.placeholder_with_default(False, shape=[])
model = net.Net(is_training)
deblur = blurs + model.generate(blurs)

In [26]:
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=4))
sess.run(tf.global_variables_initializer())

In [27]:
# Load model
print("Restoring model from " + mfile )
ut.loadNet(mfile,model.weights,sess)
print("Done!")

Restoring model from wts/supervised/iter_024000.model.npz
Done!


In [28]:
img_paths = blr_dfrm['img_path']
labels = blr_dfrm['label']
fd= dict(zip(names,img_paths.values))

In [29]:
### run for deblur
noisy, preds = sess.run([blurs, deblur], feed_dict=fd)

In [30]:
data_root_deblur = 'test_data/val_128/deblur_spvd_imgs'
labels = blr_dfrm['label']
i=0
for folder in tqdm(labels):
    dest_deblur_dir = os.path.join(data_root_deblur,folder)
    if not os.path.exists(dest_deblur_dir):
        os.makedirs(dest_deblur_dir)
    image_pth = os.path.join(dest_deblur_dir, blr_dfrm['img_id'].values[i])
    arr = np.clip(preds[i], 0, 1)
    plt.imsave(image_pth,arr)
    i=i+1

100%|██████████| 160/160 [00:00<00:00, 585.82it/s]
